In [1]:
from model.model import *
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pickle
from dataset import ShapeNetDataset
import numpy as np
import math
%matplotlib inline
from IPython.display import clear_output
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
#load data
filehandler = open('ModelNet40ProcessedDataset', 'rb') 
ShapeNetData = pickle.load(filehandler)

# print(len(ModelNet40Data.train['pcds']))
def sample_data(data,batch_size):
    total_N = len(data['pcds'])
    indices = np.random.choice(total_N,batch_size,replace = False)
    N_pts,d =  data['pcds'][0].shape
    pcds = np.zeros((batch_size,N_pts,d))
    labels = np.zeros(batch_size,dtype=int)
    for i in range(batch_size):
        pcd = data['pcds'][indices[i]]
        #rotate pointcloud
        a = np.random.rand()*math.pi
        R = np.array([[math.cos(a),-math.sin(a),0.],\
              [math.sin(a),math.cos(a),0.],\
              [0.,0.,1.]])
        pcd = pcd.dot(R)
        #jitter
        pcd += np.random.normal(0,0.02,size=pcd.shape)      
        pcds[i,:,:] = pcd
        labels[i] = int(data['labels'][indices[i]])
    return pcds,labels
    
def get_data(data):
    total_N = 600 #len(data['pcds'])
    indices = np.arange(total_N)
    N_pts,d =  data['pcds'][0].shape
    pcds = np.zeros((total_N,N_pts,d))
    labels = np.zeros(total_N,dtype=int)
    for i in range(total_N):
        #rotate and jitter pointcloud
        pcds[i,:,:] = data['pcds'][indices[i]]
        labels[i] = int(data['labels'][indices[i]])
    return pcds,labels

def compute_accuracy(preds,target):
    tmp = preds.max(1).indices == target
    return torch.sum(tmp).detach().cpu().numpy()/target.size(0)


In [3]:
##Hyperparameters
N_CLASSES = 16
EPOCHS = 1000#2000
BATCH_SIZE = 32
INIT_LR = 0.001
MOMENTUM = 0.9
LR_STEP = 20
SCHEDULER_GAMMA = 0.5
TEST_EVERY = 1
REG_WEIGHT = 0.001
criterion = nn.CrossEntropyLoss()

In [ ]:
class_ID = 0
N_of_parts = ShapeNetData.partNumber(class_ID)
net = PointNetDenseClassification(N_of_parts).to(device)
optimizer = optim.Adam(params=net.parameters(), lr=INIT_LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=LR_STEP, gamma=SCHEDULER_GAMMA)


In [ ]:
##training loop
train_losses = []
train_iterations = []
test_losses = []
test_iterations = []


for epoch in range(EPOCHS):
    optimizer.zero_grad()
    net.train()
    #load the batch of data
    data,labels = sample_data(ModelNet40Data.train,BATCH_SIZE)
    data = torch.from_numpy(data).float().to(device)
    labels = torch.from_numpy(labels).to(device)
    
    #compute the loss
    preds,M2 = net(data)
    loss = criterion(preds,labels)
    
    #add transformation matrix regularization loss
    I = torch.eye(64).unsqueeze(0).to(device)
    loss2 = torch.mean(torch.norm(torch.bmm(M2,M2.transpose(2,1)) - I, dim=(1,2)))    
    loss += REG_WEIGHT*loss2
    
    train_losses.append(loss.detach().cpu())
    train_iterations.append(epoch)
    train_accuracy = compute_mIoU(preds,labels)
    #step the optimizer
    
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    if epoch%TEST_EVERY == 0:
        with torch.no_grad():
            net.eval()
            #load the batch of test data (batch size couldn't be too big)
            data,labels = sample_data(ModelNet40Data.test,32)
            data = torch.from_numpy(data).float().to(device)
            labels = torch.from_numpy(labels).to(device)
            
            preds,M2 = net(data)
            test_loss = criterion(preds,labels)
            #add transformation matrix regularization loss
            I = torch.eye(64).unsqueeze(0).to(device)
            test_loss2 = torch.mean(torch.norm(torch.bmm(M2,M2.transpose(2,1)) - I, dim=(1,2)))    
            test_loss += REG_WEIGHT*test_loss2

            test_losses.append(test_loss.detach().cpu())
            test_iterations.append(epoch)
            test_accuracy = compute_mIoU(preds,labels)
            print('Epoch:',epoch, ';train and test accuracies:',train_accuracy,test_accuracy)

#     clear_output()
    plt.plot(train_iterations, train_losses, 'b',test_iterations, test_losses, 'r')
    plt.xlabel('Epoch')
    plt.ylabel('Loss') 
    plt.legend(['Train','Test'])
    plt.title('Epoch vs Loss')
#     plt.show()
    plt.savefig("./cls_losses.png") # save graph for training visualization

Epoch: 0 ;train and test accuracies: 0.0 0.03125
Epoch: 1 ;train and test accuracies: 0.0 0.03125
Epoch: 2 ;train and test accuracies: 0.03125 0.0625
Epoch: 3 ;train and test accuracies: 0.03125 0.0625
Epoch: 4 ;train and test accuracies: 0.03125 0.0
Epoch: 5 ;train and test accuracies: 0.03125 0.09375
Epoch: 6 ;train and test accuracies: 0.03125 0.03125
Epoch: 7 ;train and test accuracies: 0.0 0.0
Epoch: 8 ;train and test accuracies: 0.03125 0.03125
Epoch: 9 ;train and test accuracies: 0.0625 0.03125
Epoch: 10 ;train and test accuracies: 0.09375 0.125
Epoch: 11 ;train and test accuracies: 0.1875 0.09375
Epoch: 12 ;train and test accuracies: 0.125 0.03125
Epoch: 13 ;train and test accuracies: 0.125 0.03125
Epoch: 14 ;train and test accuracies: 0.0625 0.0
Epoch: 15 ;train and test accuracies: 0.125 0.0
Epoch: 16 ;train and test accuracies: 0.1875 0.03125
Epoch: 17 ;train and test accuracies: 0.15625 0.0625
Epoch: 18 ;train and test accuracies: 0.125 0.0625
Epoch: 19 ;train and test accu

Epoch: 157 ;train and test accuracies: 0.0625 0.09375
Epoch: 158 ;train and test accuracies: 0.125 0.1875
Epoch: 159 ;train and test accuracies: 0.125 0.125
Epoch: 160 ;train and test accuracies: 0.25 0.0625
Epoch: 161 ;train and test accuracies: 0.1875 0.1875
Epoch: 162 ;train and test accuracies: 0.25 0.03125
Epoch: 163 ;train and test accuracies: 0.125 0.0625
Epoch: 164 ;train and test accuracies: 0.09375 0.0
Epoch: 165 ;train and test accuracies: 0.21875 0.0
Epoch: 166 ;train and test accuracies: 0.09375 0.03125
Epoch: 167 ;train and test accuracies: 0.09375 0.15625
Epoch: 168 ;train and test accuracies: 0.15625 0.0625
Epoch: 169 ;train and test accuracies: 0.09375 0.0625
Epoch: 170 ;train and test accuracies: 0.15625 0.0625
Epoch: 171 ;train and test accuracies: 0.15625 0.125
Epoch: 172 ;train and test accuracies: 0.15625 0.125
Epoch: 173 ;train and test accuracies: 0.1875 0.125
Epoch: 174 ;train and test accuracies: 0.15625 0.09375
Epoch: 175 ;train and test accuracies: 0.15625 0

Epoch: 311 ;train and test accuracies: 0.125 0.03125
Epoch: 312 ;train and test accuracies: 0.125 0.0
Epoch: 313 ;train and test accuracies: 0.1875 0.0625
Epoch: 314 ;train and test accuracies: 0.15625 0.15625
Epoch: 315 ;train and test accuracies: 0.125 0.0625
Epoch: 316 ;train and test accuracies: 0.1875 0.03125
Epoch: 317 ;train and test accuracies: 0.125 0.15625
Epoch: 318 ;train and test accuracies: 0.09375 0.0625
Epoch: 319 ;train and test accuracies: 0.09375 0.21875
Epoch: 320 ;train and test accuracies: 0.1875 0.0
Epoch: 321 ;train and test accuracies: 0.15625 0.03125
Epoch: 322 ;train and test accuracies: 0.1875 0.09375
Epoch: 323 ;train and test accuracies: 0.125 0.125
Epoch: 324 ;train and test accuracies: 0.1875 0.09375
Epoch: 325 ;train and test accuracies: 0.0625 0.09375
Epoch: 326 ;train and test accuracies: 0.09375 0.125
Epoch: 327 ;train and test accuracies: 0.21875 0.15625
Epoch: 328 ;train and test accuracies: 0.21875 0.09375
Epoch: 329 ;train and test accuracies: 0.

Epoch: 465 ;train and test accuracies: 0.125 0.0
Epoch: 466 ;train and test accuracies: 0.15625 0.0625
Epoch: 467 ;train and test accuracies: 0.09375 0.09375
Epoch: 468 ;train and test accuracies: 0.03125 0.125
Epoch: 469 ;train and test accuracies: 0.21875 0.0625
Epoch: 470 ;train and test accuracies: 0.0625 0.0625
Epoch: 471 ;train and test accuracies: 0.25 0.125
Epoch: 472 ;train and test accuracies: 0.21875 0.0625
Epoch: 473 ;train and test accuracies: 0.125 0.09375
Epoch: 474 ;train and test accuracies: 0.1875 0.09375
Epoch: 475 ;train and test accuracies: 0.3125 0.03125
Epoch: 476 ;train and test accuracies: 0.09375 0.09375
Epoch: 477 ;train and test accuracies: 0.125 0.03125
Epoch: 478 ;train and test accuracies: 0.15625 0.09375
Epoch: 479 ;train and test accuracies: 0.25 0.03125
Epoch: 480 ;train and test accuracies: 0.1875 0.09375
Epoch: 481 ;train and test accuracies: 0.09375 0.0625
Epoch: 482 ;train and test accuracies: 0.25 0.09375
Epoch: 483 ;train and test accuracies: 0.1

Epoch: 619 ;train and test accuracies: 0.25 0.09375
Epoch: 620 ;train and test accuracies: 0.1875 0.0625
Epoch: 621 ;train and test accuracies: 0.09375 0.125
Epoch: 622 ;train and test accuracies: 0.1875 0.15625
Epoch: 623 ;train and test accuracies: 0.1875 0.09375
Epoch: 624 ;train and test accuracies: 0.09375 0.125
Epoch: 625 ;train and test accuracies: 0.25 0.15625
Epoch: 626 ;train and test accuracies: 0.1875 0.09375
Epoch: 627 ;train and test accuracies: 0.25 0.0625
Epoch: 628 ;train and test accuracies: 0.0625 0.09375
Epoch: 629 ;train and test accuracies: 0.0625 0.0625
Epoch: 630 ;train and test accuracies: 0.21875 0.125
Epoch: 631 ;train and test accuracies: 0.09375 0.125
Epoch: 632 ;train and test accuracies: 0.125 0.15625
Epoch: 633 ;train and test accuracies: 0.125 0.125
Epoch: 634 ;train and test accuracies: 0.125 0.09375
Epoch: 635 ;train and test accuracies: 0.1875 0.09375
Epoch: 636 ;train and test accuracies: 0.21875 0.0625
Epoch: 637 ;train and test accuracies: 0.125 0